# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Paillasse d'expérimentation
---

## Sujet : Import RecueilLabel

---


## Initialisation de la paillasse 
---

In [1]:
cd ../..

/Volumes/Geek/Work/Patat


In [2]:
import importlib
import pandas as pd

## Observations et environnement
---

### Chargement du Recueil Brut

In [3]:
filename = 'imports/221007-RecueilBrut.csv'

In [4]:
df_brut = pd.read_csv(f'data/{filename}')

FileNotFoundError: [Errno 2] No such file or directory: 'data/imports/221007-RecueilBrut.csv'

In [ ]:
df_brut

### Chargement Labels

In [ ]:
df_label_corpus = pd.read_csv('data/corpus/221007-LabelCorpus.csv')

In [ ]:
#df_labels.drop(columns='Unnamed: 0').to_csv('data/corpus/221007-Label.csv',index=False)

In [ ]:
df_label_corpus

## Expérience
---

### Renommage Labels

In [ ]:
import patat.imp.labels
importlib.reload(patat.imp.labels)
patat.imp.labels.label_short_names
df_brut = df_brut.rename(columns = patat.imp.labels.label_short_names)

In [ ]:
df_brut

In [ ]:
### Normalisation des labels

import patat.imp.labels

label_names = patat.imp.labels.labels

for label in label_names:
    print(label,df_brut[label].isna().sum())

In [ ]:
import numpy as np

def normalize_label(value):
    if type(value) != str:
        return np.NaN
    elif value == 'oui':
        return 1
    elif value =='non' :
        return 0
    else:
        return np.NaN

In [ ]:
for label in label_names:
    df_brut[label]=df_brut[label].apply(normalize_label)

df_brut = df_brut.convert_dtypes()

for label in label_names:
    print(label,df_brut[label].isna().sum())

In [ ]:
df_brut

### Récupération labels du corpus

In [ ]:
from patat.imp.labels import labels

In [ ]:
labels

In [ ]:
label_values = []
for index,row in df_brut.iterrows():
    for label in labels:
        if pd.notna(row[label]):
            lvalue = {
                'url' : row['url'],
                'label' : label,
                'value' : row[label],
                'owner' : 'recueil',
            }
            label_values.append(lvalue)

In [ ]:
label_values

In [ ]:
df_label_recueil = pd.DataFrame(label_values)

In [ ]:
df_label_recueil.to_csv('data/corpus/221007-LabelRecueil.csv',index=False)

In [ ]:
df_label_recueil

### Merge avec LabelCorpus

In [ ]:
# A écrire...

In [ ]:
df_label_recueil.to_csv('data/corpus/221007-LabelCorpus.csv',index=False)

## Mesure et observation des résultats
---

## Sauvegarde des résultats
---

## Conclusions
---

- Format Labl.csv : OK

## Bricolages
---

In [ ]:
df_app = pd.read_csv('data/corpus/221003-CorpusApp.csv')

In [ ]:
df_app.keys()

In [ ]:
predictor_name = '220930-tf_lr.pp'

In [ ]:
df_label_pred = df_app[['url','y_proba']].copy()

In [ ]:
df_label_pred = df_label_pred.rename(columns={"y_proba": "value"})

In [ ]:
df_label_pred['label']='infox'
df_label_pred['owner']=predictor_name

df_label_pred

### Merge

In [ ]:
df_label_merge = pd.concat([df_label_recueil, df_label_pred],ignore_index=True)

In [ ]:
df_label_merge

In [ ]:
df_label_merge.to_csv('data/corpus/221007-LabelCorpus.csv')

### ArticleCorpus

In [ ]:
df_article=df_app[df_app['article'].notna()].copy()

In [ ]:
df_article=df_article[df_article['title'].notna()]

In [ ]:
df_article=df_article.drop(columns=['infox','y_proba'])

In [ ]:
new_urls = []
for url in list(df_brut['url']):
    if url not in list(df_article['url']):
        new_urls.append(url)

In [ ]:
len(new_urls)

In [ ]:
import patat.imp.importer
import patat.imp.parser

In [ ]:
imp = patat.imp.importer.Importer()
parser = patat.imp.parser.Parser()

In [ ]:
articles = []
for url in new_urls:
    html = imp.get_webpage(url)
    site = imp.get_site(url)
    article = parser.parse_article(url,html,site)
    articles.append(article)


In [ ]:
df_articles=pd.DataFrame(articles)

In [ ]:
df_articles[df_articles['title'].isna()].value_counts('site')